In [ ]:
import pandas as pd
import numpy as np
import ast
import re
import random

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk import pos_tag
# from nltk.stem import WordNetLemmatizer

In [ ]:
# read data
pd.set_option('display.max_columns', None)
restaurants_data = pd.read_csv('restaurants_final.csv')
restaurants_data.drop('Unnamed: 0', axis=1, inplace=True) ## remove after data correction
print(restaurants_data.head())
review_data = pd.read_csv('review_final.csv')
print(review_data.head())
users = pd.read_csv('PA_restaurant_user_with_loc.csv')
print(users.head())

In [ ]:
def convert_list_str(text):
    try:
        items = ast.literal_eval(text)
        return ' '.join(items) if isinstance(items, list) else text
    except Exception:
        return text

def create_full_description(df, text_features):
    # Fill missing values
    # features = text_features + bool_features
    features = text_features
    df[features] = df[features].fillna('')
    
    # Fill missing editorial_summary with types + categories (without commas)
    df['editorial_summary'] = df['editorial_summary'].apply(lambda x: x if x.strip() != '' else '')
    # df['types'] = df['types'].str.replace(',', ' ')
    df['categories'] = df['categories'].str.replace(',', ' ')
    df.loc[df['editorial_summary'].str.strip() == '', 'editorial_summary'] = (
        df['categories']
    )
    
    # Convert list-formatted column to plain text
    # df[list_feature] = df[list_feature].apply(convert_list_str)

    # Remove commas from all text features
    for col in text_features:
        df[col] = df[col].str.replace(',', ' ')
    
    # if len(bool_features) != 0:
    #     # Convert boolean columns to descriptive tags
    #     for col in bool_features:
    #         df[col] = df[col].apply(lambda x: f"{col.lower()}" if x == True else "")
        
    # Combine all into a single description column
    df['description'] = df[text_features]\
        .agg(' '.join, axis=1)\
        .str.replace(r'\s+', ' ', regex=True)\
        .str.strip()

    return df

In [ ]:
item_df = restaurants_data.copy() 
# text_features = ['categories', 'editorial_summary', 'types', 'city']
text_features = ['categories', 'editorial_summary', 'city', 'name']
# bool_features = [
#             'OutdoorSeating', 'RestaurantsTakeOut', 'RestaurantsDelivery',
#             'RestaurantsReservations', 'GoodForKids', 'RestaurantsGoodForGroups',
#             'HappyHour', 'serves_breakfast', 'serves_lunch', 'serves_dinner',
#             'serves_brunch', 'serves_beer', 'serves_wine', 'serves_cocktails',
#             'serves_dessert', 'serves_coffee', 'BusinessAcceptsCreditCards', 'HasTV',
#             'BikeParking', 'Caters', 'RestaurantsTableService',
#         ]
bool_features = []
# list_features = ['parking_options']
list_features = []
# handle WiFi column
# handle RestaurantsAttire column
# handle BusinessParking column
# handle NoiseLevel column
# handle Ambience column
# handle GoodForMeal column
# remove WheelchairAccessible column
# handle Alcohol column
# handle DogsAllowed column



item_df = create_full_description(item_df, text_features)
# remove text features
item_df = item_df.drop(text_features, axis=1)
print(item_df[['business_id', 'description']].head())
print(item_df.head())


In [ ]:
bool_features = [
            'OutdoorSeating', 'RestaurantsTakeOut', 'RestaurantsDelivery',
            'RestaurantsReservations', 'GoodForKids', 'RestaurantsGoodForGroups',
            'serves_breakfast', 'serves_lunch', 'serves_dinner',
            'serves_brunch', 'serves_beer', 'serves_wine', 'serves_cocktails',
            'serves_dessert', 'serves_coffee', 'BusinessAcceptsCreditCards',
        ]

# check for missing values in bool features
print(item_df[bool_features].isnull().sum())
# we remove HappyHour, HasTV, BikeParking, Caters, RestaurantsTableService due to the high number of missing values
# confirm that these columns only contain true/false values
print(item_df[bool_features].describe())
# fill missing values with False (we assume that if the value is not present, it is False)
item_df[bool_features] = item_df[bool_features].fillna(False)
# convert bool features to int
for col in bool_features:
    item_df[col] = item_df[col].astype(int)

print(item_df[bool_features].describe()) # confirm that these columns only contain 0/1 values


In [ ]:
# select only the columns we need
# item_df = item_df[['business_id', 'description'] + bool_features]
item_df = item_df[['business_id'] + bool_features]
print(item_df.shape)
print(item_df.columns)
print(item_df.head())

In [ ]:
review_df = review_data.copy()
review_df['interaction'] = review_df['stars'].apply(lambda x: 1 if x >= 3 else 0)
interaction_df = review_df[['user_id', 'business_id', 'stars', 'interaction', 'date']].copy()
print(interaction_df.head(2))

In [ ]:
numerical_user_features = ['review_count', 'useful', 'funny', 'cool', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile',
                           'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_writer', 'compliment_cool', 
                           'compliment_funny', 'compliment_photos', "('latitude', 'mean')", "('longitude', 'mean')"]
# select only the columns we need
user_df = users.copy()
# not yet handled latitude and longitude
user_df = user_df[['user_id'] + numerical_user_features]
print(user_df.shape)

# standardize numerical features to between 0 and 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
user_df[numerical_user_features] = scaler.fit_transform(user_df[numerical_user_features])
print(user_df.head(2))
print(user_df.describe())

In [ ]:
# random sample 100 users
user_df = user_df.sample(1000, random_state=42)

# combine user, restaurant and review data
user_restaurant_review = pd.merge(interaction_df, item_df, on='business_id', how='left')
user_restaurant_review = pd.merge(user_restaurant_review, user_df, on='user_id', how='right')
print(user_restaurant_review.shape)
print(user_restaurant_review.describe())
print(user_restaurant_review.head(2))

# remove name, yelping_since, 

In [ ]:
import os
import urllib.request
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split


In [ ]:
class RestaurantRatingModel(nn.Module):
    def __init__(self, num_users, num_restaurants, embedding_dim=32):
        super(RestaurantRatingModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.restaurant_embedding = nn.Embedding(num_restaurants, embedding_dim)
        self.mlp = nn.Sequential(
            nn.Linear(2*embedding_dim, 256), # 2x embedding_dim because we concatenate user and item embeddings
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 1)
        )

    def forward(self, user, restaurant):
        user_embedding = self.user_embedding(user)
        restaurant_embedding = self.restaurant_embedding(restaurant)
        concat = torch.cat([user_embedding, restaurant_embedding], dim=1)
        output = self.mlp(concat)
        return output

In [ ]:
# class RestaurantRatingModel(nn.Module):
#     def __init__(self, num_users, num_restaurants, user_num_features, restaurant_num_features, embedding_dim=50):
#         super(RestaurantRatingModel, self).__init__()
        
#         # User tower components
#         self.user_embedding = nn.Embedding(num_users, embedding_dim)
#         self.user_fc = nn.Linear(embedding_dim + user_num_features, 128)
        
#         # Restaurant tower components
#         self.restaurant_embedding = nn.Embedding(num_restaurants, embedding_dim)
#         self.restaurant_fc = nn.Linear(embedding_dim + restaurant_num_features, 128)

#         # Review tower components
#         self.review_embedding = nn.Embedding(num_reviews, embedding_dim)
#         self.review_fc = nn.Linear(embedding_dim + review_num_features, 128)
        
#         # Combined MLP
#         self.combined_fc1 = nn.Linear(256, 256)
#         self.combined_fc2 = nn.Linear(256, 128)
#         self.output = nn.Linear(128, 1)
        
#         # Activation function
#         self.relu = nn.ReLU()
        
#     def forward(self, user_id, user_numerical, restaurant_id, restaurant_numerical):
#         # User tower
#         user_emb = self.user_embedding(user_id).squeeze(1)
#         user_combined = torch.cat([user_emb, user_numerical], dim=1)
#         user_out = self.relu(self.user_fc(user_combined))
        
#         # Restaurant tower
#         restaurant_emb = self.restaurant_embedding(restaurant_id).squeeze(1)
#         restaurant_combined = torch.cat([restaurant_emb, restaurant_numerical], dim=1)
#         restaurant_out = self.relu(self.restaurant_fc(restaurant_combined))
        
#         # Combine towers
#         merged = torch.cat([user_out, restaurant_out], dim=1)
#         x = self.relu(self.combined_fc1(merged))
#         x = self.relu(self.combined_fc2(x))
#         output = self.output(x)
        
#         return output

# You'll need to create a custom Dataset class for your data
class RestaurantDataset(Dataset):
    def __init__(self, dataframe):
        self.user_ids = torch.tensor(dataframe['user_id_encoded'].values, dtype=torch.long)
        # self.user_numerical = torch.FloatTensor(user_numerical)
        self.restaurant_ids = torch.tensor(dataframe['business_id_encoded'].values, dtype=torch.long)
        # self.restaurant_numerical = torch.FloatTensor(restaurant_numerical)
        self.ratings = torch.tensor(dataframe['interaction'].values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.user_ids[idx], self.restaurant_ids[idx], self.ratings[idx]


In [ ]:
# encode user_id and restaurant_id
from sklearn.preprocessing import LabelEncoder

user_encoder = LabelEncoder()
restaurant_encoder = LabelEncoder()

user_restaurant_review['user_id_encoded'] = user_encoder.fit_transform(user_restaurant_review['user_id'])
user_restaurant_review['business_id_encoded'] = restaurant_encoder.fit_transform(user_restaurant_review['business_id'])

train_df = user_restaurant_review[user_restaurant_review['date'] < '2021-01-01']
test_df = user_restaurant_review[user_restaurant_review['date'] >= '2021-01-01']

# remove date column
train_df.drop(['date'], axis=1, inplace=True)
test_df.drop(['date'], axis=1, inplace=True)

train_dataset = RestaurantDataset(train_df)
test_dataset = RestaurantDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# train_X = train_df.drop(['stars', 'interaction'], axis=1)
# train_y = train_df['interaction']
# # train_y = train_df['stars']
# # take only 5000 samples for training
# train_X = train_X.sample(n=5000, random_state=42)
# train_y = train_y[train_X.index]

# # test_X = test_df.drop(['user_id', 'business_id', 'stars', 'interaction', 'date'], axis=1)
# test_X = test_df.drop(['stars', 'interaction'], axis=1)
# test_y = test_df['interaction']
# # test_y = test_df['stars']
# print(train_X.head(2))

In [ ]:
# Example usage:
# Assuming you have prepared your data
num_users = len(np.unique(user_restaurant_review['user_id_encoded']))
num_restaurants = len(np.unique(user_restaurant_review['business_id_encoded']))
user_num_features = len(numerical_user_features)
restaurant_num_features = len(bool_features)

model = RestaurantRatingModel(
    num_users=num_users,
    num_restaurants=num_restaurants,
    # user_num_features=user_num_features,
    # restaurant_num_features=restaurant_num_features,
    embedding_dim= 64
)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Example training loop
def train(model, dataloader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for batch in dataloader:
            user, item, label = batch
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(user, item)
            loss = criterion(outputs.squeeze(), label)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')
        evaluate_model(model, test_loader)

def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            user, item, label = batch
            outputs = model(user, item)
            predicted = (outputs.squeeze() >= 0.5).float()  # Assuming binary classification
            total += label.size(0)
            correct += (predicted == label).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# Train the model
train(model, train_loader, criterion, optimizer, num_epochs=10)

In [ ]:
# history = model.fit(
#     [train_X['user_id'], train_X[user_num_features], train_X['business_id'], train_X[restaurant_num_features]],
#     train_y,
#     validation_data=([test_X['user_id'], test_X[user_num_features], test_X['business_id'], test_X[restaurant_num_features]], test_y),
#     epochs=10, batch_size=64
# )

In [ ]:
# def recommend_top_restaurants(user_id, model, restaurants_data, user_features):
#     user_data = user_features[user_features['user_id'] == user_id].iloc[0]
#     user_inputs = np.array([user_id]), user_data[user_num_features].values.reshape(1, -1)
    
#     restaurant_scores = []
#     for _, restaurant in restaurants_data.iterrows():
#         restaurant_inputs = np.array([restaurant['business_id']]), restaurant[restaurant_num_features].values.reshape(1, -1)
#         score = model.predict([user_inputs[0], user_inputs[1], restaurant_inputs[0], restaurant_inputs[1]])
#         restaurant_scores.append((restaurant['business_id'], score))
    
#     # Sort and select top 10
#     restaurant_scores.sort(key=lambda x: x[1], reverse=True)
#     return restaurant_scores[:10]

# # Example usage
# # top_restaurants = recommend_top_restaurants(user_id="X88LNRDRZ_1TooB71AIgpA", model=model, restaurants_data=item_df, user_features=user_df)
# # print("Top 10 Recommendations:", top_restaurants)

# def evaluate_model(model, test_X, test_y):
#     predictions = model.predict([test_X['user_id'], test_X[user_numerical_features], test_X['business_id'], test_X[restaurant_numerical_features]])

In [ ]:
# run evaluations with NDCG@10, Precision@10

import numpy as np
from sklearn.metrics import ndcg_score, precision_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve


def evaluate_model_final(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    labels = []
    with torch.no_grad():
        for batch in test_loader:
            user, item, label = batch
            outputs = model(user, item)
            predicted = (outputs.squeeze() >= 0.5).float()  # Assuming binary classification
            total += label.size(0)
            correct += (predicted == label).sum().item()
            predictions.extend(outputs.squeeze().numpy())
            labels.extend(label.numpy())

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
    rmse = np.sqrt(mean_squared_error(labels, predictions))
    print(f"RMSE: {rmse:.4f}")
    ndcg = ndcg_score([labels], [predictions], k=10)
    print(f"NDCG@10: {ndcg:.4f}")
    # precision = precision_score([labels], [predictions])
    # print(f"Precision@10: {precision:.4f}")


    # predictions = model.predict([test_X['user_id'], test_X['business_id']])
    # predictions = predictions.flatten()
    # rmse = np.sqrt(mean_squared_error(test_y, predictions))
    # print(f"RMSE: {rmse:.4f}")
    # ndcg = ndcg_score([test_y], [predictions], k=10)
    # print(f"NDCG@10: {ndcg:.4f}")
    # precision = precision_score([test_y], [predictions], k=10)
    # print(f"Precision@10: {precision:.4f}")
    # return rmse, ndcg, precision

evaluate_model_final(model, test_loader)

# rmse = evaluate_model(model, test_X, test_y)
# print(f"RMSE: {rmse:.4f}")

# # calculate NDCG@10
# ndcg = ndcg_score([test_y], [predictions], k=10)
# print(f"NDCG@10: {ndcg:.4f}")

# # calculate Precision@10
# precision = precision_score([test_y], [predictions], k=10)
# print(f"Precision@10: {precision:.4f}")


In [ ]:
def get_recommendation(user_id, k=10):
    model.eval()
    user_restaurant_review_specific = user_restaurant_review[user_restaurant_review['user_id'] == user_id]
    dataset = RestaurantDataset(user_restaurant_review_specific)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

    # get predictions for all restaurants
    restaurant_scores = []
    with torch.no_grad():
        for batch in dataloader:
            user, item, _ = batch
            outputs = model(user, item)
            restaurant_scores.append((item.item(), outputs.item()))

    # sort by predicted rating
    restaurant_scores.sort(key=lambda x: x[1], reverse=True)
    # get top k restaurants
    recommended_restaurants = restaurant_scores[:k]
    # convert to DataFrame
    recommended_restaurants_df = pd.DataFrame(recommended_restaurants, columns=['business_id_encoded', 'predicted_rating'])
    # add business_id
    recommended_restaurants_df['business_id'] = restaurant_encoder.inverse_transform(recommended_restaurants_df['business_id_encoded'])
    return recommended_restaurants_df

def hits_at_k(user_id, test_df, recommend_func, k=10):
    # true_item = test_df.loc[test_df['user_id'] == user_id]['business_id'].iloc[0]
    true_items = test_df[(test_df['user_id'] == user_id) &
                         # Hit rate should consider if the user liked the restaurant
                         (test_df['interaction'] == 1)
                         ]['business_id'].tolist()
    recommended = recommend_func(user_id, k)

    recommended_items = recommended['business_id'].tolist()

    return int(any(item in recommended_items for item in true_items))

def plot_hit_rate_vs_users(test_df, max_users, step):
    # Step 1: Get users sorted by interaction count descending
    user_interaction_counts = (
        test_df.groupby('user_id')
        .size()
        .sort_values(ascending=False)
    )
    sorted_users = user_interaction_counts.index.tolist()


    # Step 2: Loop with cumulative hit calculation
    hit_rates = []
    total_hits = 0
    evaluated_users = 0

    for i in range(step, max_users + 1, step):
        current_batch = sorted_users[evaluated_users:i]
        hits_in_batch = sum(
            hits_at_k(user, test_df, get_recommendation, k=10)
            for user in current_batch
        )
        total_hits += hits_in_batch
        evaluated_users = i
        hit_rate = total_hits / evaluated_users
        hit_rates.append(hit_rate)

    # Step 3: Plot
    n_users_range = list(range(step, max_users + 1, step))
    plt.figure(figsize=(10, 6))
    print(n_users_range, hit_rates)
    plt.plot(n_users_range, hit_rates, marker='o', linestyle='-', color='b')
    plt.xlabel("Number of Top Users (by Interactions)")
    plt.ylabel("Hit Rate@10")
    plt.title("Hit Rate vs. Number of Included Users")
    plt.grid(True)
    plt.show()

In [22]:
# torch.set_default_tensor_type(torch.FloatTensor)  # force CPU tensors
# model = model.cpu()

plot_hit_rate_vs_users(test_df, max_users=1000, step=100)

: 